In [1]:
#Replication Project Code

In [2]:
import pandas as pd

In [3]:
data = pd.read_stata("Downloads/AER20090377_FinalData.dta")

In [4]:
neighbor_data = pd.read_stata("Downloads/112465-V1/AER20090377_DataAndPrograms/AER20090377_NeighborData.dta")

In [5]:
# Set memory, processors, and other configurations (not directly equivalent in pandas)
# These configurations are often not necessary in Python, as memory management and parallel processing are typically handled differently.

# Load the data
final_data = data.copy()

# Data preparation
# Keep years before 2004
final_data = final_data[final_data['year'] < 2004]

# Drop rows with invalid observations
final_data = final_data[final_data['valid'] >= 9]

# Keep only Jun-Aug months
final_data = final_data[final_data['month'].isin([6, 7, 8])]


In [6]:
num_days_per_year = final_data.groupby(['fips', 'site_id', 'year']).size().reset_index(name='NumDays')
num_days_per_year = num_days_per_year[num_days_per_year['NumDays'] >= 69]
num_days_per_year.head()

,fips,site_id,year,NumDays
0,1001.0,3,1990.0,92
1,1003.0,10,2000.0,89
2,1003.0,10,2001.0,91
3,1003.0,10,2002.0,89
4,1003.0,10,2003.0,88


In [7]:
# Keep monitor-years with at least 75% of days with observations
final_data = final_data.merge(num_days_per_year[['fips', 'site_id', 'year', 'NumDays']], on=['fips', 'site_id', 'year'], how='right')

In [8]:
# Sort the DataFrame
final_data.sort_values(by=['fips', 'site_id', 'year', 'ozone_max'], inplace=True)

# Collapse to get the mean ozone_max by fips, site_id, and year
temp_data = final_data.groupby(['fips', 'site_id', 'year'])['ozone_max'].mean().reset_index().copy()

In [9]:
# Group by 'fips' and 'site_id' and count the number of unique years
final_data['NumYear'] = final_data.groupby(['fips', 'site_id'])['year'].transform('count')


In [10]:
# Drop duplicate rows based on 'fips', 'site_id', and 'year' columns
final_data_no_duplicates = final_data.drop_duplicates(subset=['fips', 'site_id', 'year'])

# Display the first few rows of the DataFrame without duplicates
print(final_data_no_duplicates.head())

counts = final_data.groupby(['fips', 'site_id', 'year']).size()

# Filtering combinations with 15 or fewer occurrences
filtered_counts = counts[counts <= 15]

# Getting the total number of rows
total_rows = filtered_counts.sum()

sorted_counts = counts.sort_values()
print(sorted_counts)




     state_code  county_code  site_id  valid   epa_8hr  ozone_max   day  \
1             1            1        3   11.0  0.016714      0.020   2.0   
108           1            3       10   12.0  0.018250      0.019  18.0   
247           1            3       10   12.0  0.018750      0.021   7.0   
295           1            3       10   12.0  0.017875      0.020  27.0   
397           1            3       10   12.0  0.013000      0.015   7.0   

     month    year       Date  ...  NumOffMax NumOffMin NumOff1Max NumOff1Min  \
1      6.0  1990.0 1990-06-02  ...        0.0       0.0        0.0        0.0   
108    6.0  2000.0 2000-06-18  ...        0.0       1.0        0.0        1.0   
247    8.0  2001.0 2001-08-07  ...        0.0       1.0        0.0        1.0   
295    6.0  2002.0 2002-06-27  ...        0.0       1.0        0.0        1.0   
397    7.0  2003.0 2003-07-07  ...        2.0       1.0        2.0        1.0   

    NOtherStationprcp _merge urban  _mergeurb NumDays  NumYear

In [11]:
# Group by 'fips' and 'site_id' and get unique 'year' values for each group
year_table = final_data.groupby(['fips', 'site_id'])['year'].unique().reset_index()

# Count the number of years in each row
year_table['num_years'] = year_table['year'].apply(len)

year_table.head()

year_table_filtered = year_table[year_table['num_years'] == 15]


filtered_final_data = final_data.merge(year_table_filtered[['fips', 'site_id']], on=['fips', 'site_id'], how='right')

In [12]:
full_history_monitors = filtered_final_data.to_stata('FullHistoryMonitors.dta')

In [13]:
# Sort both DataFrames by 'fips' and 'site_id'
temp_data.sort_values(by=['fips', 'site_id'], inplace=True)
filtered_final_data.sort_values(by=['fips', 'site_id'], inplace=True)

# Merge the datasets on 'fips' and 'site_id'
new_data = pd.merge(temp_data, filtered_final_data, on=['fips', 'site_id'], how='right')

In [22]:
filtered_final_data['ozone_max']

0         0.017
1         0.018
2         0.019
3         0.022
4         0.028
          ...  
469851    0.083
469852    0.083
469853    0.084
469854    0.088
469855    0.105
Name: ozone_max, Length: 469856, dtype: float32

In [14]:
# Sort main_data by 'fips'
new_data.sort_values(by='fips', inplace=True)

# Merge main_data with neighbor_data on 'fips'
neighbor_data = neighbor_data.merge(new_data[['fips', 'site_id', 'year']], on=['fips'], how='inner')

KeyError: "['year'] not in index"

In [ ]:
neighbor_data.head()

In [ ]:
# Turn off RFG when CARB is on
merged_data.loc[merged_data['treat_CARB'] != 0, 'treat_rfg'] = 0

temp_data = merged_data

In [ ]:
# Generate interaction terms
temp_data.loc[:, '_DMTempMax'] = temp_data['DOW'] * temp_data['TempMax']
temp_data.loc[:, '_DmTempMin'] = temp_data['DOW'] * temp_data['TempMin']
temp_data.loc[:, '_DrRain'] = temp_data['DOW'] * temp_data['Rain']
temp_data.loc[:, '_DsSnow'] = temp_data['DOW'] * temp_data['Snow']

# Generate 'DOY' (day of year)
temp_data.loc[:, 'DOY'] = temp_data['Date'].dt.dayofyear

#Temperature Polynomials
temp_data.loc[:, 'TempMax1'] = temp_data['TempMax']
temp_data.loc[:, 'TempMax2'] = temp_data['TempMax'] * temp_data['TempMax']
temp_data.loc[:, 'TempMax3'] = temp_data['TempMax'] * temp_data['TempMax'] * temp_data['TempMax']
temp_data.loc[:, 'TempMin1'] = temp_data['TempMin']
temp_data.loc[:, 'TempMin2'] = temp_data['TempMin'] * temp_data['TempMin']
temp_data.loc[:, 'TempMin3'] = temp_data['TempMin'] * temp_data['TempMin'] * temp_data['TempMin']
temp_data.loc[:, 'TempMaxMin'] = temp_data['TempMin'] * temp_data['TempMax']
temp_data.loc[:, 'Rain1'] = temp_data['Rain']
temp_data.loc[:, 'Rain2'] = temp_data['Rain'] * temp_data['Rain']
temp_data.loc[:, 'Snow1'] = temp_data['Snow']
temp_data.loc[:, 'Snow2'] = temp_data['Snow'] * temp_data['Snow']
temp_data.loc[:, 'RainTempMax'] = temp_data['Rain'] * temp_data['TempMax']



In [ ]:
temp_data = temp_data.sort_values(by=['fips', 'site_id', 'Date'])
temp_data['TempMaxL1'] = temp_data.groupby(['fips', 'site_id'])['TempMax'].shift(1)
temp_data.loc[temp_data.groupby(['fips', 'site_id'])['TempMaxL1'].head(1).index, 'TempMaxL1'] = np.nan
temp_data['TempMinL1'] = temp_data.groupby(['fips', 'site_id'])['TempMin'].shift(1)
temp_data.loc[temp_data.groupby(['fips', 'site_id'])['TempMinL1'].head(1).index, 'TempMinL1'] = np.nan
temp_data['TempMaxMaxL1'] = temp_data['TempMax'] * temp_data['TempMaxL1']
temp_data['TempMaxMinL1'] = temp_data['TempMax'] * temp_data['TempMinL1']

In [ ]:
start_index = temp_data.columns.get_loc('TempMax1')
end_index = temp_data.columns.get_loc('TempMaxMinL1') + 1
for var in temp_data.columns[start_index:end_index]:
    temp_data['DOY' + var] = temp_data['DOY'] * temp_data[var]

# Keep only Jun-Aug
temp_data = temp_data[temp_data['month'].isin([6, 7, 8])]

# Save the modified data
temp_data.to_stata(file_path + "temp.dta", write_index=False)

In [ ]:
#Additional variables: income. Take logs of ozone

income_data = pd.read_stata("Downloads/112465-V1/AER20090377_DataAndPrograms/AER20090377_IncomeData.dta")

income_data.head()

In [ ]:
temp_data.head()

In [ ]:
temp_data.sort_values(by=['state_code', 'county_code', 'year'], inplace=True)

# Merge temp_data with income_data on state_code, county_code, and year
merged_data = pd.merge(temp_data, income_data, on=['state_code', 'county_code', 'year'], how='inner', suffixes=('_temp', '_income'))

merged_data.head()

In [ ]:
# Drop rows where ozone_max or epa_8hr is equal to 0
merged_data = merged_data[(merged_data['ozone_max'] != 0) & (merged_data['epa_8hr'] != 0)]

# Drop rows where any of the TempMax1-TempMax3 or TempMin1-TempMin3 variables is missing
temp_max_vars = [f"TempMax{i}" for i in range(1, 4)]
temp_min_vars = [f"TempMin{i}" for i in range(1, 4)]
merged_data = merged_data.dropna(subset=temp_max_vars + temp_min_vars)

# Drop rows where income is missing
merged_data = merged_data.dropna(subset=['income'])

merged_data['lozone_max'] = np.log10(merged_data['ozone_max'])
merged_data['lozone_8hr'] = np.log10(merged_data['epa_8hr'])


In [ ]:
#Create census region dummies and interactions. Create time trend and interactions

merged_data['region'] = np.nan
merged_data['division'] = np.nan



In [ ]:
state_region_mapping = {
    1: 3, 2: 4, 4: 4, 5: 3, 6: 4, 8: 4, 9: 1, 10: 3, 11: 3, 12: 3,
    13: 3, 15: 4, 16: 4, 17: 2, 18: 2, 19: 2, 20: 2, 21: 3, 22: 3, 23: 1,
    24: 3, 25: 1, 26: 2, 27: 2, 28: 3, 29: 2, 30: 4, 31: 2, 32: 4, 33: 1,
    34: 1, 35: 4, 36: 1, 37: 3, 38: 2, 39: 2, 40: 3, 41: 4, 42: 1, 44: 1,
    45: 3, 46: 2, 47: 3, 48: 3, 49: 4, 50: 1, 51: 3, 53: 4, 54: 3, 55: 2, 56: 4
}

# Update the 'region' column based on the state_code
merged_data.loc[merged_data['state_code'] == 1, 'region'] = 3
merged_data.loc[merged_data['state_code'] == 2, 'region'] = 4
# Repeat the above line for each state_code and corresponding region using the state_region_mapping dictionary

# You can also use a loop to iterate over the state_region_mapping dictionary
for state_code, region in state_region_mapping.items():
    merged_data.loc[merged_data['state_code'] == state_code, 'region'] = region
    

In [ ]:
merged_data.loc[:, '_RY'] = merged_data['year']*merged_data['region']
merged_data.loc[:, '_RW'] = merged_data['DOW']*merged_data['region']
merged_data.loc[:, '_RD'] = merged_data['DOY']*merged_data['region']
merged_data['DateS'] = merged_data['Date'].dt.dayofyear / 365
merged_data['DateS2'] = (merged_data['DateS'])**2

In [ ]:
# Drop the specified columns (commented out on repeat)
merged_data.drop(columns=['RVPCty', 'RFGCty', 'CARBCty'], inplace=True)

# Group by 'fips' and create new variables 'RVPCty' and 'RFGCty' within each group
merged_data['RVPCty'] = merged_data.groupby('fips')['treat_rvpII'].transform('max')
merged_data['RFGCty'] = merged_data.groupby('fips')['treat_rfg'].transform('max')


In [ ]:
merged_data.loc[:, 'RVPRFGCty'] = 0
# Replace RVPRFGCty with 1 where both RFGCty and RVPCty are equal to 1
merged_data.loc[(merged_data['RFGCty'] == 1) & (merged_data['RVPCty'] == 1), 'RVPRFGCty'] = 1

# Replace RFGCty with 0 where RVPRFGCty is equal to 1
merged_data.loc[merged_data['RVPRFGCty'] == 1, 'RFGCty'] = 0

# Replace RVPCty with 0 where RVPRFGCty is equal to 1
merged_data.loc[merged_data['RVPRFGCty'] == 1, 'RVPCty'] = 0

In [ ]:
# Assuming merged_data is a pandas DataFrame
merged_data['CARBCty'] = merged_data.groupby('fips')['treat_CARB'].transform('max')
merged_data.loc[:, 'CARBRFGCty'] = 0

# Replace CARBRFGCty with 1 if either RFGCty or RVPRFGCty is 1 and CARBCty is 1
merged_data['CARBRFGCty'] = ((merged_data['RFGCty'] == 1) | (merged_data['RVPRFGCty'] == 1)) & (merged_data['CARBCty'] == 1)

# Replace RFGCty with 0 if CARBRFGCty is 1

merged_data.loc[merged_data['CARBRFGCty'] == 1, 'RFGCty'] = 0

merged_data.loc[merged_data['CARBRFGCty'] == 1, 'RVPRFGCty'] = 0

merged_data.loc[merged_data['CARBRFGCty'] == 1, 'CARBCty'] = 0

In [ ]:
for i in range(1, 5):
    merged_data[f'TrendRVP{i}'] = 0
    merged_data.loc[(merged_data['RVPCty'] == 1) & (merged_data['region'] == i), f'TrendRVP{i}'] = merged_data['DateS']
for i in range(1, 5):
    merged_data[f'TrendRFG{i}'] = 0
    merged_data.loc[(merged_data['RFGCty'] == 1) & (merged_data['region'] == i), f'TrendRFG{i}'] = merged_data['DateS']
for i in range(1, 5):
    merged_data[f'TrendRVPRFG{i}'] = 0
    merged_data.loc[(merged_data['RVPRFGCty'] == 1) & (merged_data['region'] == i), f'TrendRVPRFG{i}'] = merged_data['DateS']
for i in range(1, 5):
    merged_data[f'TrendCARB{i}'] = 0
    merged_data.loc[(merged_data['CARBCty'] == 1) & (merged_data['region'] == i), f'TrendCARB{i}'] = merged_data['DateS']  
for i in range(1, 5):
    merged_data[f'TrendCARBRFG{i}'] = 0
    merged_data.loc[(merged_data['CARBRFGCty'] == 1) & (merged_data['region'] == i), f'TrendCARBRFG{i}'] = merged_data['DateS']    

In [ ]:
for i in range(1, 5):
    merged_data[f'QTrendRVP{i}'] = 0
    merged_data.loc[(merged_data['RVPCty'] == 1) & (merged_data['region'] == i), f'QTrendRVP{i}'] = merged_data['DateS']**2

for i in range(1, 5):
    merged_data[f'QTrendRFG{i}'] = 0
    merged_data.loc[(merged_data['RFGCty'] == 1) & (merged_data['region'] == i), f'QTrendRFG{i}'] = merged_data['DateS']**2
    
for i in range(1, 5):
    merged_data[f'QTrendRVPRFG{i}'] = 0
    merged_data.loc[(merged_data['RVPRFGCty'] == 1) & (merged_data['region'] == i), f'QTrendRVPRFG{i}'] = merged_data['DateS']**2
    
for i in range(1, 5):
    merged_data[f'QTrendCARB{i}'] = 0
    merged_data.loc[(merged_data['CARBCty'] == 1) & (merged_data['region'] == i), f'QTrendCARB{i}'] = merged_data['DateS']**2
    
for i in range(1, 5):
    merged_data[f'QTrendCARBRFG{i}'] = 0
    merged_data.loc[(merged_data['CARBRFGCty'] == 1) & (merged_data['region'] == i), f'QTrendCARBRFG{i}'] = merged_data['DateS']**2


In [ ]:
# Sort the DataFrame by 'state_code' and 'year'
merged_data.sort_values(by=['state_code', 'year'], inplace=True)

# Generate a new variable 'StateYear' based on grouping by 'state_code' and 'year'
merged_data['StateYear'] = merged_data.groupby(['state_code', 'year']).ngroup()

# Save the DataFrame to a file
merged_data.to_stata('DD_AnalysisDataset_NYR.dta', write_index=False, convert_dates={'Date': 'td'})


In [ ]:
#The REGRESSION!

#Step one is to first-difference everything on panelid

# Mean-difference all variables on panelid
merged_data['A_fips'] = merged_data['fips']
merged_data['A_month'] = merged_data['month']
merged_data['A_year'] = merged_data['year']
merged_data['A_StateYear'] = merged_data['StateYear']
merged_data['A_StateCode'] = merged_data['state_code']
merged_data['A_EstTemp'] = merged_data['EstTempFlag']
merged_data['A_EstTempPrcp'] = merged_data['EstTempFlagprcp']

# Keep selected columns
merged_data = merged_data.filter(regex='^lozone_|^treat|^Temp|^Rain|^Snow|^DOY|^Trend|^QTrend|^DateS|_D|_R|income|panelid|^A_')

# Sort by panelid
merged_data.sort_values(by='panelid', inplace=True)


In [ ]:
import pandas as pd

# List to store generated columns
generated_columns = []

# Iterate over selected columns
for var in merged_data.filter(regex='^lozone_|^treat|^Temp|^Rain|^Snow|^DOY|^Trend|^QTrend|^DateS|_D|_R|income').columns:
    # Calculate mean by panelid
    merged_data[f'M{var}'] = merged_data.groupby('panelid')[var].transform('mean')
    # Generate differenced variable
    generated_columns.append(merged_data[var] - merged_data[f'M{var}'])

# Concatenate all generated columns at once
merged_data = pd.concat([merged_data] + generated_columns, axis=1)

# Drop original and mean columns
merged_data.drop(columns=merged_data.filter(like='M').columns, inplace=True)

# Save to a new file
merged_data.to_stata('DD_AnalysisDataset_Diffed_NYR.dta', write_index=False)


In [ ]:
# Wait now actually the regressions!

import statsmodels.api as sm

# Define independent and dependent variables
X = merged_data.filter(regex='^treat').copy()  # Independent variables
y = merged_data['lozone_maxD']  # Dependent variable

# Add constant term if needed
# X = sm.add_constant(X)

# Perform regression with clustered standard errors
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Print regression summary
print(model.summary())

# Test parameter equality
hypotheses = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
print(model.t_test(hypotheses).summary())

# Output regression results to a text file
with open('DDResults_NYR.txt', 'w') as f:
    f.write(model.summary().as_text())


In [ ]:
# Define independent and dependent variables
X = merged_data.filter(regex='^treat|_RY').copy()  # Independent variables
y = merged_data['lozone_maxD']  # Dependent variable

# Perform regression with clustered standard errors
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Print regression summary
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model.summary().as_text() + '\n\n')  # Append results to text file

# Test parameter equality
hypotheses = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model.t_test(hypotheses).summary().as_text() + '\n\n')  # Append results to text file

In [ ]:
# Regression 1: Adding DOY and DOW controls
X1 = merged_data.filter(regex='^treat|_R').copy()  # Independent variables
y1 = merged_data['lozone_maxD']  # Dependent variable

model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses1 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model1.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 2: Adding weather and weather cross effects
X2 = merged_data.filter(regex='^treat|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y2 = merged_data['lozone_maxD']  # Dependent variable

model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses2 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model2.t_test(hypotheses2).summary().as_text() + '\n\n')  # Append results to text file


In [ ]:
# Regression 1: Adding income
X1 = merged_data.filter(regex='^treat|income|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y1 = merged_data['lozone_maxD']  # Dependent variable

model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses1 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model1.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 2: Adding linear trends
X2 = merged_data.filter(regex='^treat|income|Trend|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y2 = merged_data['lozone_maxD']  # Dependent variable

model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses2 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model2.t_test(hypotheses2).summary().as_text() + '\n\n')  # Append results to text file


# Regression 3: Adding quadratic trends
X3 = merged_data.filter(regex='^treat|income|Trend|QTrend|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y3 = merged_data['lozone_maxD']  # Dependent variable

model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses3 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model3.t_test(hypotheses3).summary().as_text() + '\n\n')  # Append results to text file


In [ ]:
#Looking at the dependent variable epa_8hr

# Regression 1: Base regression
X1 = merged_data.filter(regex='^treat').copy()  # Independent variables
y1 = merged_data['lozone_8hrD']  # Dependent variable

model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
hypotheses1 = 'treat_rvpIID = treat_rfgD, treat_rvpIID = treat_CARBD, treat_rfgD = treat_CARBD'
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model1.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 2: Adding year * region FE
X2 = merged_data.filter(regex='^treat|_RY').copy()  # Independent variables
y2 = merged_data['lozone_8hrD']  # Dependent variable

model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model2.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 3: Adding DOY and DOW controls
X3 = merged_data.filter(regex='^treat|_R').copy()  # Independent variables
y3 = merged_data['lozone_8hrD']  # Dependent variable

model3 = sm.OLS(y3, X3).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model3.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 4: Adding weather and weather cross effects
X4 = merged_data.filter(regex='^treat|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y4 = merged_data['lozone_8hrD']  # Dependent variable

model4 = sm.OLS(y4, X4).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model4.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 5: Adding income
X5 = merged_data.filter(regex='^treat|income|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y5 = merged_data['lozone_8hrD']  # Dependent variable

model5 = sm.OLS(y5, X5).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model5.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 6: Adding linear trends
X6 = merged_data.filter(regex='^treat|income|Trend|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y6 = merged_data['lozone_8hrD']  # Dependent variable

model6 = sm.OLS(y6, X6).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model6.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file


# Regression 7: Adding quadratic trends
X7 = merged_data.filter(regex='^treat|income|Trend|QTrend|Temp|Rain|Snow|DOY|_D|_R').copy()  # Independent variables
y7 = merged_data['lozone_8hrD']  # Dependent variable

model7 = sm.OLS(y7, X7).fit(cov_type='cluster', cov_kwds={'groups': merged_data['A_StateYear']})

# Test parameter equality
with open('DDResults_NYR.txt', 'a') as f:
    f.write(model7.t_test(hypotheses1).summary().as_text() + '\n\n')  # Append results to text file



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_stata('DD_AnalysisDataset_Diffed.dta')

# Regress ozone on weather and seasonal/day effects
# Assuming you've already performed this regression and obtained the residuals
# The residuals are stored in the column 'O3_Resid'

# Create a county type variable
data['CountyType'] = 0
data.loc[data['A_RVPCty'] == 1, 'CountyType'] = 1
data.loc[(data['A_RFGCty'] == 1) | (data['A_RVPRFGCty'] == 1), 'CountyType'] = 2
data.loc[(data['A_CARBCty'] == 1) | (data['A_CARBRFGCty'] == 1), 'CountyType'] = 3

# Collapse for plotting
plot_data = data.groupby(['CountyType', 'A_year'])['O3_Resid'].mean().reset_index()

# Plot RVP vs Baseline
plt.figure(figsize=(10, 6))
for ctype in [1, 0]:
    plt.plot(plot_data[plot_data['CountyType'] == ctype]['A_year'],
             plot_data[plot_data['CountyType'] == ctype]['O3_Resid'],
             label=f'County Type {ctype}')
plt.title("Summer ozone concentrations, RVP vs. baseline counties")
plt.xlabel("Year")
plt.ylabel("Average daily maximum ozone, ppm")
plt.grid(True)
plt.legend()
plt.savefig('Graph_AnnualResids_RVP.png')
plt.show()

# Plot RFG vs Baseline
plt.figure(figsize=(10, 6))
for ctype in [2, 0]:
    plt.plot(plot_data[plot_data['CountyType'] == ctype]['A_year'],
             plot_data[plot_data['CountyType'] == ctype]['O3_Resid'],
             label=f'County Type {ctype}')
plt.title("Summer ozone concentrations, RFG vs. baseline counties")
plt.xlabel("Year")
plt.ylabel("Average daily maximum ozone, ppm")
plt.grid(True)
plt.legend()
plt.savefig('Graph_AnnualResids_RFG.png')
plt.show()

# Plot CARB vs Baseline
plt.figure(figsize=(10, 6))
for ctype in [3, 0]:
    plt.plot(plot_data[plot_data['CountyType'] == ctype]['A_year'],
             plot_data[plot_data['CountyType'] == ctype]['O3_Resid'],
             label=f'County Type {ctype}')
plt.title("Summer ozone concentrations, CARB vs. baseline counties")
plt.xlabel("Year")
plt.ylabel("Average daily maximum ozone, ppm")
plt.grid(True)
plt.legend()
plt.savefig('Graph_AnnualResids_CARB.png')
plt.show()
